<a href="https://colab.research.google.com/github/SubhamMohanty187/Data-Science/blob/main/Python/Predict_rating_of_product_reviews_on_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Amazon Food Review Analysis

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.

Number of reviews: 568,454

Number of users: 256,059

Number of products: 74,258

Timespan: Oct 1999 - Oct 2012

Number of Attributes/Columns in data: 10

Attribute Information:

Id

ProductId - unique identifier for the product

UserId - unqiue identifier for the user
ProfileName

HelpfulnessNumerator - number of users who found the review helpful

HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not

Score - rating between 1 and 5

Time - timestamp for the review

Summary - brief summary of the review

Text - text of the review


##Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).


[Q] How to determine if a review is positive or negative?

[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.

In [ ]:
! gdown --id 1NYlMiKOoQVwkXgY_bdesnQUjKnanzxbl

Downloading...
From: https://drive.google.com/uc?id=1NYlMiKOoQVwkXgY_bdesnQUjKnanzxbl
To: /content/Reviews.csv
100% 301M/301M [00:01<00:00, 198MB/s]


In [ ]:
import sqlite3                  # Library to Use sqLite
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk                     # NLP ToolKit library for Text Processing
import re                       # Library for Regular Expression in String


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer   # (BoW aaproach)  Sci-kit learn libraray that Converts texts to vectors

In [ ]:
from nltk.corpus import stopwords  # Gives us a set of stopword in english language.
from nltk.stem import PorterStemmer 
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
 
# for tsne assignment you can take 5k data points

data = pd.read_csv("Reviews.csv")
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
print(data.shape)

(568454, 10)


In [ ]:
df = data.copy()
# df_p = df.loc[df["Score"] > 3]  # df_p contains data with score > 3
# df_n = df.loc[df["Score"] < 3]  # df_n contains data with score < 3

df = df.loc[df["Score"] != 3]

In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


NOTE-: 

Time is stored in UNIX Time stamp.

In [ ]:
# Give reviews with Score>3 a positive rating (i.e 1), and reviews with a score <3 a negative rating(i.e 0).
def partition(x):
    if x < 3:
        return 0
    return 1

df["Score"] = df["Score"].map(partition)  # map() helps to apply a fun() to all items in a list.
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
print(df.shape)  # Original Shape is (568454, 10).

(525814, 10)


##Exploratory Data Analysis

####Data Cleaning: Deduplication 

It is observed (as shown in the table below) that the reviews data has many duplicate entries. Hence it is necessary to remove duplicates in order to get unbiased results for the analysis of the data.

In [ ]:
# duplicate = df[df.duplicated("UserId")]  # Use df.duplicated() to find duplicate items in our column on data.
# duplicate
df.loc[df["UserId"] == "AR5J8UI46CURR"]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
73790,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,1,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
78444,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,1,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
138276,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,1,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
138316,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,1,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
155048,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,1,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As we can see that there are duplicate items in our dataset having same values for Time , Reviews, Summary etc. so we have to remove these.

As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [ ]:
# Sort the dataset by productId
sorted_df = df.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
# Find and remove duplicate entries in our data
final_df = sorted_df.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep="first", inplace=False)
# Since, Only UserID,ProfileName,Time,Text have the duplicate items we only remove items from those columns and keep="fisrt" 
# means keep the first value and delete the rest of the duplicate items
final_df.shape

(364173, 10)

In [ ]:
# To see how much of our original data still remains
(final_df['Id'].size*1.0)/(df['Id'].size*1.0)*100

69.25890143662969

In [ ]:
final_df.loc[final_df["HelpfulnessNumerator"] > final_df["HelpfulnessDenominator"]]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,1,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,1,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


As we know that HelpfulnessNumerator(+ve) should always be less than or equal to HelpfulnessDenominator(+ve. + -ve.). So, we should remove these incorrect data.

In [ ]:
final_df = final_df.loc[final_df["HelpfulnessNumerator"] <= final_df["HelpfulnessDenominator"]]

In [ ]:
print(final_df.shape)                     # Final no. of entries left.
print(final_df["Score"].value_counts())   # No. of positive and negative reviews.

(364171, 10)
1    307061
0     57110
Name: Score, dtype: int64


##Text Preprocessing.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

###Bag of Words

In [ ]:
count_vect = CountVectorizer()
final_df_count = count_vect.fit_transform(final_df["Text"].values)

In [ ]:
type(final_df_count)    #sparse matrix is a matrix in which most of the values are 0.

scipy.sparse.csr.csr_matrix

In [ ]:
final_df_count.get_shape()    # Matrix with 36k rows and 115k vectors (represnting each sentence of 36k rows of reviews)

(364171, 115281)

Regular Expression

In [ ]:
                 
stop_word = set(stopwords.words("english"))  #gives a set of stopwords
sno = SnowballStemmer("english")  # initializing the snowball stemmer to english language.

In [ ]:
print(sno.stem("tasty"))

tasti
